In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotPromptTemplate, FewShotChatMessagePromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ],
)

examples = [
    {
    "movie": "Inception",
    "answer": """
    Director: Christopher Nolan
    Main Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page
    Budget: $160 million
    Box Office: $836.8 million
    Genre: Sci-Fi, Thriller
    Synopsis: A group of experts who specialize in extracting secrets from within dreams take on an almost impossible task: planting an idea into a target's subconscious. As they delve deeper, the line between reality and dreams blurs.
"""
}, 
{
    "movie": "The Grand Budapest Hotel",
    "answer": """
    Director: Wes Anderson
    Main Cast: Ralph Fiennes, Tony Revolori, Saoirse Ronan
    Budget: $25 million
    Box Office: $172.9 million
    Genre: Comedy, Drama
    Synopsis: Set in 1930s Europe, the story follows the adventures of a famous hotel's concierge and a young lobby boy who become embroiled in a murder mystery involving a wealthy guest, a stolen painting, and a family fortune.
"""
},
{
    "movie": "The Social Network",
    "answer": """
    Director: David Fincher
    Main Cast: Jesse Eisenberg, Andrew Garfield, Justin Timberlake
    Budget: $40 million
    Box Office: $224.9 million
    Genre: Drama, Biographical
    Synopsis: The film chronicles the rise of Mark Zuckerberg from a Harvard student to the co-founder of Facebook. As the social network grows, so do the personal and legal battles, exposing the costs of ambition and innovation.
"""
}]

example_template = """
human: Do you know about {movie}?
ai: {answer}
"""
example_prompt = PromptTemplate.from_template(example_template)

example_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Do you know about {movie}?",
    input_variables=["movie"]
)

chain = example_prompt | chat
chain.invoke({
    "movie":"The Dark Knight"
})


    Director: Christopher Nolan
    Main Cast: Christian Bale, Heath Ledger, Aaron Eckhart
    Budget: $185 million
    Box Office: $1.005 billion
    Genre: Action, Crime, Drama
    Synopsis: Batman faces his greatest challenge yet as he battles the Joker, a sadistic criminal mastermind who seeks to create chaos in Gotham City. The Dark Knight must confront his own inner demons while trying to save the city from destruction.


'\nexample_template = """\nhuman: Do you know about {movie}?\nAI: {answer}\n"""\n\nexample_prompt = PromptTemplate.from_template(example_template)\n\nprompt = FewShotPromptTemplate(\n    example_prompt=example_prompt,\n    examples=examples,\n    suffix="Do you know about {movie}?",\n    input_variables=["movie"]\n)\n\nmovie_chain = prompt | chat\n\nmovie_chain.invoke({\n    "movie":"Interstellar"\n})\n'